In [1]:
import pandas as pd

train = pd.read_csv('C:/python/signate/data/train.csv', index_col=0)

In [2]:
# 'zip_city'と'boroname'の組み合わせを作成
train['combined_address'] = train['zip_city'] + ', ' + train['boroname']

# ユニークな組み合わせを抽出
unique_addresses = train[['zip_city', 'boroname']].drop_duplicates()
unique_addresses['combined_address'] = unique_addresses['zip_city'] + ', ' + unique_addresses['boroname']

In [3]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import tqdm

# ジオコーディングの実行
geolocator = Nominatim(user_agent="geoapiExercises")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

unique_addresses['location'] = unique_addresses['combined_address'].apply(lambda addr: geocode(addr))
unique_addresses['latitude'] = unique_addresses['location'].apply(lambda loc: loc.latitude if loc else None)
unique_addresses['longitude'] = unique_addresses['location'].apply(lambda loc: loc.longitude if loc else None)

RateLimiter caught an error, retrying (0/2 tries). Called with (*('Little Neck, Queens',), **{}).
Traceback (most recent call last):
  File "c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\geopy\geocoders\base.py", line 368, in _call_geocoder
    result = self.adapter.get_json(url, timeout=timeout, headers=req_headers)
  File "c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\geopy\adapters.py", line 472, in get_json
    resp = self._request(url, timeout=timeout, headers=headers)
  File "c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\geopy\adapters.py", line 500, in _request
    raise AdapterHTTPError(
geopy.adapters.AdapterHTTPError: Non-successful status code 403

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\koba_\AppData\Local\Programs\Python\Python310\lib\site-packages\geopy\extra\rate_limiter.py", line 136, in _retries_gen
    yield

In [4]:
train = pd.merge(train, unique_addresses[['combined_address', 'latitude', 'longitude']], on='combined_address', how='left')

In [5]:
import folium

# 地図の基本設定（中心座標と初期ズームレベルを設定）
m = folium.Map(location=[40.7128, -74.0060], zoom_start=11)  # ニューヨーク市を中心に設定

# データポイントを地図上にプロット
for index, row in train.iterrows():
    # 緯度と経度が存在する行のみプロット
    if pd.notnull(row['latitude']) and pd.notnull(row['longitude']):
        # healthの値に基づいて色を決定
        if row['health'] == 1:
            marker_color = 'green'
        elif row['health'] == 2:
            marker_color = 'red'
        else:
            marker_color = 'blue'

        # マーカーを追加
        folium.CircleMarker(
            location=(row['latitude'], row['longitude']),
            radius=5,
            color=marker_color,
            fill=True,
            fill_opacity=0.7
        ).add_to(m)

ModuleNotFoundError: No module named 'folium'

In [ ]:
m